In [1]:
# Firstly, we must import some libs
import os
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import json
import yaml
import matplotlib.pyplot as plt

import torch
from PIL import Image
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
import shutil as sh

import json
from collections import Counter

In [2]:
colab = 'google.colab' in str(get_ipython())
colab

True

In [3]:
if colab:
    p_dir = '/content'
    from google.colab import drive
    drive.mount('/content/drive')
else:
    p_dir = '/home/zyp/w/d2l/detect_outfits'


Mounted at /content/drive


In [4]:
if colab:
    !git clone https://github.com/garrisonz/yolov5.git
    !pip install -r yolov5/requirements.txt # install dependencies
    
    #显示运行时间
    !pip install ipython-autotime
    %load_ext autotime
    
    # load data
    src = '/content/drive/MyDrive/colab/detect_outfits/cowboyoutfits.zip'
    dst = '/content/data/cowboyoutfits.zip'
    dir = '/content/data/origin/'
    os.makedirs(dir, exist_ok=True)
    sh.copy(src, dst)
    sh.unpack_archive(dst, dir)
    len(next(os.walk(dir))[2])

Cloning into 'yolov5'...
remote: Enumerating objects: 12132, done.
remote: Total 12132 (delta 0), reused 0 (delta 0), pack-reused 12132
Receiving objects: 100% (12132/12132), 11.89 MiB | 25.05 MiB/s, done.
Resolving deltas: 100% (8399/8399), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 7.5 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 19.2 s (started: 2022-06-07 23:26:56 +00:00)


In [7]:
if colab:
    history_dir = p_dir + '/history'
    os.makedirs(history_dir, exist_ok=True)

    basic_epochs = 40
    total_epochs = basic_epochs
    archive_name = f'bln_epochs{basic_epochs}'
    sh.copy(f'/content/drive/MyDrive/colab/detect_outfits/log/{archive_name}.zip', history_dir)

    sh.unpack_archive(f'{history_dir}/{archive_name}.zip' , history_dir)
    basic_pt = history_dir + '/' + archive_name + '/weights/last.pt'
    print(basic_pt) 
    

time: 354 ms (started: 2022-06-07 23:28:22 +00:00)


In [9]:
json_file_path = p_dir + '/data/origin/train.json'
data = json.load(open(json_file_path, 'r'))

time: 27.9 ms (started: 2022-06-07 23:29:09 +00:00)


In [10]:
cats = pd.DataFrame(data["categories"])[['id', 'name']].set_index('id')
cats_dict = cats.squeeze().to_dict()
cats_dict

{87: 'belt', 131: 'boot', 318: 'cowboy_hat', 588: 'jacket', 1034: 'sunglasses'}

time: 21.8 ms (started: 2022-06-07 23:29:11 +00:00)


In [11]:
anns = pd.DataFrame(data['annotations'])[['id', 'image_id', 'category_id', 'bbox']].set_index("id")
anns

,image_id,category_id,bbox
id,,,
12550146,15526467552013451612,1034,"[102.49, 181.12, 137.08, 97.92]"
9764874,12017556593931260822,318,"[284.8, 297.6, 252.8, 192.0]"
13729810,17157301591781950087,318,"[431.36, 277.25, 471.04, 314.39]"
13959212,17494374403857745282,1034,"[195.76, 237.59, 363.09, 85.33]"
12910638,15979914912395313233,131,"[263.04, 368.64, 182.4, 238.72]"
...,...,...,...
8388588,10275527987851298058,1034,"[284.16, 0.0, 739.2, 682.36]"
1277936,1379797987660853607,318,"[470.4, 49.6, 262.4, 211.2]"
12943352,16021391580251818415,588,"[105.6, 203.2, 916.8, 563.2]"


time: 24.1 ms (started: 2022-06-07 23:31:58 +00:00)


In [12]:
Counter(anns['category_id'])

Counter({87: 25, 131: 449, 318: 595, 588: 2195, 1034: 2330})

time: 5.2 ms (started: 2022-06-07 23:31:59 +00:00)


In [13]:
def cc2yolo_bbox(img_width, img_height, bbox):
    dw = 1. / img_width
    dh = 1. / img_height
    x = bbox[0] + bbox[2] / 2.0
    y = bbox[1] + bbox[3] / 2.0
    w = bbox[2]
    h = bbox[3]
 
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

def yolo2cc_bbox(img_width, img_height, bbox):
    x = (bbox[0] - bbox[2] * 0.5) * img_width
    y = (bbox[1] - bbox[3] * 0.5) * img_height
    w = bbox[2] * img_width
    h = bbox[3] * img_height
    
    return (x, y, w, h)

def cat_to_id():
    cat_id_map = {}
    num = 0
    for cate in data['categories']:
        cat_id_map[cate['id']] = num
        num+=1
    return cat_id_map

cat_id_map = cat_to_id()
cate_id_map = cat_id_map
print(cat_id_map)

re_cat_id_map = dict(zip(cat_id_map.values(), cat_id_map.keys()))
print(re_cat_id_map)

{87: 0, 1034: 1, 131: 2, 318: 3, 588: 4}
{0: 87, 1: 1034, 2: 131, 3: 318, 4: 588}
time: 17.2 ms (started: 2022-06-07 23:31:59 +00:00)


In [14]:
imgs = pd.DataFrame(data['images'])[["id", "file_name", "width", "height"]].set_index('id')

small_set = 0
if small_set:
    imgs = imgs[:10]
imgs

,file_name,width,height
id,,,
9860841628484337660,88d8bf3754317ffc.jpg,1024,681
15984033263460081658,ddd2b190ea90dffa.jpg,768,1024
76077631043502082,010e4833cdb38002.jpg,1024,683
18065680256228130812,fab6307a1a43fffc.jpg,683,1024
9491379842992996352,83b827ae01e68000.jpg,1024,768
...,...,...,...
10412147159402939302,907f60acf14e03a6.jpg,1024,819
4866906309960860041,438ab980c0530189.jpg,1024,681
10441816352675791240,90e8c8a64f1b0188.jpg,1024,1024


time: 17.9 ms (started: 2022-06-07 23:31:59 +00:00)


In [15]:
train_imgs, valid_imgs = train_test_split(imgs, test_size=0.2, random_state=233)
print(len(train_imgs), len(valid_imgs))
train_imgs

2449 613


,file_name,width,height
id,,,
5180813895364543634,47e5f2c9e89c8492.jpg,1024,1001
17003661551598027224,ebf9241787f7c1d8.jpg,1024,681
12743304695942482017,b0d9501aa2380861.jpg,1024,640
1699768557964395625,1796ca7f69d88469.jpg,768,1024
9173391940436394037,7f4e6f5dfb248035.jpg,1024,683
...,...,...,...
4088288576445451227,38bc84ccb2518bdb.jpg,1024,683
12791859404168037664,b185d05a9a2d0520.jpg,768,1024
15646941784381426067,d9251aa6df458993.jpg,1024,1024


time: 12.7 ms (started: 2022-06-07 23:31:59 +00:00)


In [16]:
belt_imgs = []
for i in data['annotations']:
    if i['category_id'] == 87 and i['image_id'] in train_imgs.index:
        belt_imgs.append(i['image_id'])

print(len(belt_imgs), len(set(belt_imgs)))
belt_imgs = list(set(belt_imgs))
#belt_imgs

19 17
time: 4.13 ms (started: 2022-06-07 23:31:59 +00:00)


In [17]:
len(train_imgs.reset_index().to_numpy())

2449

time: 6.12 ms (started: 2022-06-07 23:31:59 +00:00)


In [18]:
anns

,image_id,category_id,bbox
id,,,
12550146,15526467552013451612,1034,"[102.49, 181.12, 137.08, 97.92]"
9764874,12017556593931260822,318,"[284.8, 297.6, 252.8, 192.0]"
13729810,17157301591781950087,318,"[431.36, 277.25, 471.04, 314.39]"
13959212,17494374403857745282,1034,"[195.76, 237.59, 363.09, 85.33]"
12910638,15979914912395313233,131,"[263.04, 368.64, 182.4, 238.72]"
...,...,...,...
8388588,10275527987851298058,1034,"[284.16, 0.0, 739.2, 682.36]"
1277936,1379797987660853607,318,"[470.4, 49.6, 262.4, 211.2]"
12943352,16021391580251818415,588,"[105.6, 203.2, 916.8, 563.2]"


time: 12.1 ms (started: 2022-06-07 23:31:59 +00:00)


In [19]:
def build_yolo_dataset(is_train, imgs):
    if is_train:
        t = 'train'
    else:
        t = 'valid'
    
    yolo_img_dir = f'{p_dir}/data/yolo2/{t}/images'
    yolo_label_dir = f'{p_dir}/data/yolo2/{t}/labels'
    src_img_dir = p_dir + '/data/origin/images'
    
    os.makedirs(yolo_img_dir, exist_ok=True)
    os.makedirs(yolo_label_dir, exist_ok=True)
    
    for img in imgs.reset_index().to_numpy():
        #print(img)
        img_id = img[0]
        img_name = img[1].split('.')[0]
        img_w = img[2]
        img_h = img[3]
        #print(img_name)
        
        # 复制 yolo 图像
        sh.copyfile(f'{src_img_dir}/{img_name}.jpg', f'{yolo_img_dir}/{img_name}.jpg')
        #print(f'{yolo_img_dir}/{img_name}.jpg')
        
        # 生成 yolo 标签
        with open(f'{yolo_label_dir}/{img_name}.txt', 'w') as f:
            for ann in anns.reset_index().to_numpy():
                ann_img_id = ann[1]
                ann_cid = ann[2]
                ann_bbox = ann[3]
                if img_id == ann_img_id:
                    #print(ann)
                    bbox = cc2yolo_bbox(img_w, img_h, ann_bbox)
                    f.write('{} {} {} {} {}\n'.format(cat_id_map[ann_cid], bbox[0], bbox[1], bbox[2], bbox[3]))
        #print(f'{yolo_label_dir}/{img_name}.txt')

build_yolo_dataset(is_train=True, imgs=train_imgs)
build_yolo_dataset(is_train=False, imgs=valid_imgs)

time: 10.6 s (started: 2022-06-07 23:31:59 +00:00)


In [ ]:
def repeat_belt():
    yolo_img_dir = f'{p_dir}/data/yolo2/train/images'
    yolo_label_dir = f'{p_dir}/data/yolo2/train/labels'
    src_img_dir = p_dir + '/data/origin/images'
    
    repeat = 430
    for i in range(repeat):
        suffix = int(i / len(belt_imgs)) + 1
        img_id = belt_imgs[i % len(belt_imgs)]
        img = train_imgs.loc[img_id]
        img_name = img.file_name.split('.')[0]
        new_name = f'{img_name}_{suffix}'
        sh.copyfile(f'{src_img_dir}/{img_name}.jpg', f'{yolo_img_dir}/{new_name}.jpg')
        #print(f'{yolo_img_dir}/{new_name}.jpg')
        
        # 复制 yolo 标签
        sh.copyfile(f'{yolo_label_dir}/{img_name}.txt', f'{yolo_label_dir}/{new_name}.txt')
        #print(f'{yolo_img_dir}/{new_name}.txt')
        
repeat_belt()

In [21]:
# Create  yaml file
data_yaml = dict(
    train = f'{p_dir}/data/yolo2/train/images',
    val = f'{p_dir}/data/yolo2/valid/images',
    nc = 5,
    names = ['belt', 'sunglasses', 'boot', 'cowboy_hat', 'jacket']
)

yolo_data_yaml = p_dir + '/data/yolo2/data.yaml'
with open(yolo_data_yaml, 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=True)

time: 7.3 ms (started: 2022-06-07 23:32:11 +00:00)


In [22]:
if colab:
    train_py = p_dir + '/yolov5/train.py'
else:
    train_py = '~/w/github/yolov5/train.py'
epochs = 20
total_epochs += epochs 
out_dir = p_dir + f'/log/bln_epochs{total_epochs}'

os.environ['WANDB_DISABLED'] = 'true'
!python {train_py} --img 640 --batch 32 --epochs {epochs} --data {yolo_data_yaml}   \
                        --weights {basic_pt} --name {out_dir}

train: weights=/content/history/bln_epochs20/weights/last.pt, cfg=, data=/content/data/yolo2/data.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=/content/log/bln_epochs40, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/garrisonz/yolov5 ✅
YOLOv5 🚀 v6.1-238-g0f07795 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=

In [25]:
if colab:
    %cd /content/log
    !zip -r bln_epochs{total_epochs}.zip bln_epochs{total_epochs}
    %cd /content
    !mv /content/log/bln_epochs{total_epochs}.zip /content/drive/MyDrive/colab/detect_outfits/log/

/content/log
  adding: bln_epochs40/ (stored 0%)
  adding: bln_epochs40/train_batch2.jpg (deflated 3%)
  adding: bln_epochs40/R_curve.png (deflated 8%)
  adding: bln_epochs40/labels_correlogram.jpg (deflated 26%)
  adding: bln_epochs40/F1_curve.png (deflated 8%)
  adding: bln_epochs40/val_batch0_pred.jpg (deflated 10%)
  adding: bln_epochs40/results.csv (deflated 82%)
  adding: bln_epochs40/results.png (deflated 6%)
  adding: bln_epochs40/val_batch0_labels.jpg (deflated 10%)
  adding: bln_epochs40/opt.yaml (deflated 46%)
  adding: bln_epochs40/PR_curve.png (deflated 8%)
  adding: bln_epochs40/labels.jpg (deflated 15%)
  adding: bln_epochs40/P_curve.png (deflated 7%)
  adding: bln_epochs40/val_batch2_pred.jpg (deflated 5%)
  adding: bln_epochs40/train_batch0.jpg (deflated 2%)
  adding: bln_epochs40/train_batch1.jpg (deflated 3%)
  adding: bln_epochs40/weights/ (stored 0%)
  adding: bln_epochs40/weights/best.pt (deflated 9%)
  adding: bln_epochs40/weights/last.pt (deflated 9%)
  adding: 